In [121]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [122]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [123]:
txt = open('dati_1.txt')
n = int(txt.readline())
p = int(txt.readline())
k = int(txt.readline())
print(n)
print(p)
print(k)

3
5
2


In [124]:
x = torch.empty(n,p,device=device)
y = torch.empty(n,k,device=device)

In [125]:
for i in range(0, n):
    temp1 = txt.readline()
    temp2 = [float(i) for i in temp1.split()]
    x[i,:] = torch.tensor(temp2[0:p])
    y[i,:] = torch.tensor(temp2[p:(p+k)])
txt.close()
print(x)
print(y)

tensor([[  1.,   2.,   3.,   4.,   5.],
        [  8.,   9.,  10.,  11.,  12.],
        [ 15.,  16.,  17.,  18.,  19.]], device='cuda:0')
tensor([[  6.,   7.],
        [ 13.,  14.],
        [ 20.,  21.]], device='cuda:0')


In [126]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = n, p, 3, k
model = torch.nn.Sequential(
          torch.nn.Linear(D_in, H),
          torch.nn.Tanh(),
          torch.nn.Linear(H, D_out),
).to(device)
loss_fn = torch.nn.MSELoss(size_average=False)
learning_rate = 1e-2

In [127]:
# initial weights
model.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.3173,  0.0030,  0.0195, -0.4384,  0.1024],
                      [ 0.2279,  0.0287,  0.2410,  0.1794,  0.4461],
                      [ 0.0479,  0.1539,  0.1899,  0.0775,  0.2318]], device='cuda:0')),
             ('0.bias', tensor([ 0.3095, -0.0127, -0.2906], device='cuda:0')),
             ('2.weight', tensor([[-0.3058, -0.3002, -0.5471],
                      [-0.2955,  0.3434, -0.2366]], device='cuda:0')),
             ('2.bias', tensor([-0.3913, -0.4789], device='cuda:0'))])

In [128]:
# define custom weights
W = []
W.append(torch.randn( (3,5), device='cuda:0') )
W.append(torch.randn( (3,), device='cuda:0') )
W.append(torch.randn( (2,3), device='cuda:0') )
W.append(torch.randn( (2,), device='cuda:0') )

In [129]:
print(W)

[tensor([[ 1.2471, -0.1005,  0.4805, -0.0787,  1.4565],
        [ 1.9809,  0.9995,  0.5979, -2.2009, -1.3230],
        [-3.0381,  1.4245,  0.0385, -0.3047,  0.2374]], device='cuda:0'), tensor([-0.5122, -0.1825, -0.3201], device='cuda:0'), tensor([[-0.3160, -0.0375, -1.8809],
        [-0.0487, -0.7292,  1.3571]], device='cuda:0'), tensor([-0.0196,  1.2554], device='cuda:0')]


In [130]:
# custom weights initialization      
def weights_init(model):
    count = 0
    for m in model:
        classname = m.__class__.__name__           
        if classname.find('Linear') != -1:        
            print('Init Linear module weights')
            m.weight.data = W[count]
            count += 1
            print('Init Linear module bias')
            m.bias.data = W[count]
            count += 1  

In [131]:
weights_init(model)

Init Linear module weights
Init Linear module bias
Init Linear module weights
Init Linear module bias


In [132]:
print('\n')
print(list(model.parameters()))



[Parameter containing:
tensor([[ 1.2471, -0.1005,  0.4805, -0.0787,  1.4565],
        [ 1.9809,  0.9995,  0.5979, -2.2009, -1.3230],
        [-3.0381,  1.4245,  0.0385, -0.3047,  0.2374]], device='cuda:0'), Parameter containing:
tensor([-0.5122, -0.1825, -0.3201], device='cuda:0'), Parameter containing:
tensor([[-0.3160, -0.0375, -1.8809],
        [-0.0487, -0.7292,  1.3571]], device='cuda:0'), Parameter containing:
tensor([-0.0196,  1.2554], device='cuda:0')]


In [99]:
# txt_w = open("customweights_pytorch.txt")
# temp3 = txt_w.read()
# txt_w.close()

# custom_w = [float(i) for i in temp3.split()]

# ind = 0
# for j in range(0,H):
#     params[1][j] = custom_w[ind]
#     ind += 1
#     for i in range(0, p):
#         params[0][j][i] = custom_w[ind]
#         ind += 1
# for j in range(0,k):
#     params[3][j] = custom_w[ind]
#     ind += 1
#     for i in range(0, H):
#         params[2][j][i] = custom_w[ind]
#         ind += 1

In [133]:
for t in range(1000):
      
    model.zero_grad()
    y_pred = model(x)
    
    loss = loss_fn(y_pred, y)
    loss.backward()
    
    with torch.no_grad():
        for param in model.parameters():
            param.data -= learning_rate * param.grad
    print(loss.item())

1128.87841796875
740.9195556640625
510.7197265625
377.697509765625
300.81158447265625
256.3102722167969
230.33328247070312
213.46177673339844
155.04440307617188
179.8581085205078
519.3567504882812
382.7708740234375
303.87884521484375
258.31085205078125
231.99070739746094
216.78823852539062
208.00729370117188
202.93539428710938
200.00587463378906
198.3137969970703
197.33642578125
196.7718963623047
196.44583129882812
196.25750732421875
196.14871215820312
196.08592224121094
196.04962158203125
196.02862548828125
196.01651000976562
196.00955200195312
196.0054931640625
196.00318908691406
196.00184631347656
196.00100708007812
196.00059509277344
196.0003204345703
196.0001983642578
196.0001220703125
196.00003051757812
196.0000457763672
196.00003051757812
196.0
195.99998474121094
196.0
196.0
195.99996948242188
196.0
196.0
195.99996948242188
195.99996948242188
196.0
195.99996948242188
195.99998474121094
196.0
196.0
195.99996948242188
196.0
196.0
196.0
196.0
195.99998474121094
196.0
196.0
196.0
19